# Load data

In [ ]:
import glob
import os

In [ ]:
data = []
for n in glob.glob("un/TXT/Session 77 - 2022/*.txt"):
    data.append({"country": os.path.basename(n.replace("_77_2022.txt", "")), "text": open(n).read() })

In [ ]:
import pandas as pd

In [ ]:
pd.set_option('display.max_colwidth', 500)
df = pd.DataFrame(data)
df

# Sentence segmentation

In [ ]:
import spacy

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
from tqdm.auto import tqdm

In [ ]:
# runs 30 seconds or so
sentences = []
for text in tqdm(df["text"]):
    doc = nlp(text)
    for sentence in doc.sents:
        sentences.append(str(sentence).strip())

In [ ]:
sentences[0:20]

In [ ]:
len(sentences)

In [ ]:
open("sentences.txt", "w").write("@@@".join(sentences))

# Encode sentences

In [ ]:
!pip install sentence_transformers -U

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

In [ ]:
!pip install huggingface_hub -U

In [ ]:
# can take a minute or two depending on CPU/GPU configuration
sembeddings = model.encode(sentences, show_progress_bar=True)

In [ ]:
len(sembeddings)

In [ ]:
sembeddings[0].shape

In [ ]:
import numpy as np
with open("sentences.npy", "wb") as f:
    np.save(f, sembeddings)

In [ ]:
# can take a minute or two depending on CPU/GPU configuration
model2 = SentenceTransformer('mixedbread-ai/mxbai-embed-large-v1')
sembeddings2 = model2.encode(sentences, show_progress_bar=True, normalize_embeddings=True)

In [ ]:
with open("sentences2.npy", "wb") as f:
    np.save(f, sembeddings2)

# Retrieval

In [ ]:
def search(query, text, corpus_embeddings, model, top=20):
    # code query to restrict search space
    question_embedding = model.encode(query, normalize_embeddings=True)
    
    # Determine similarity (vectors are normalized)
    sim = np.dot(corpus_embeddings, question_embedding)
    
    # Get most similar top_k by sorting
    hits = [ { "text": text[i], "score": sim[i] } 
                     for i in sim.argsort()[::-1][0:top] ]
    
    # Return as dataframe
    return pd.DataFrame(hits)

In [ ]:
pd.set_option('display.max_colwidth', 0)

In [ ]:
search("The climate crisis is worse for poorer countries.", sentences, sembeddings, model)

In [ ]:
search("The climate crisis is worse for poorer countries.", sentences, sembeddings2, model2)